In [0]:
import delta

def table_exists(catalog, database, table):
    
    count = (spark.sql(f"SHOW TABLES FROM {catalog}.{database}")
            .filter(f"database = '{database}' AND tableName = '{table}'")
            .count())
    return count == 1

In [0]:
catalog = 'bronze'
schema = 'olist_ecommerce'
tablename = dbutils.widgets.get('tablename')
id_field = dbutils.widgets.get('id_field')
timestamp_field = dbutils.widgets.get('timestamp_field')

In [0]:
# Esta camada de código tem o objetivo de fazer a ingestão de um arquivo Full Load na pasta de Bronze. 
# O arquivo Full Load é uma Tabela em formato CSV, que contém dados mas que precisam ser atualizados através de arquivos CDC (Change Data Capture).

# Nesta parte do código, precisamos primeiro ler o arquivo CSV com o Spark que logo após a leitura se transforma em um Dataframe. 
# No código podemos notar .option("header", "true"), utilizei esse comando pois com ele conseguimos nos comunicar com o Spark e dizer que o arquivo CSV contém cabeçalho, ou seja, nome de colunas presentes na primeira.

if not table_exists(catalog, schema, tablename):

  print('Tabela não existente. Criando...')


  df_full_table = spark.read.format("csv").option("header", "true").load(f"/Volumes/raw/olist_ecommerce/full_load/{tablename}/")

    # Logo após a leitura preciso salvar o Dataframe no Schema de Bronze.

  (df_full_table.coalesce(1)    # Com coalesce(1), garantimos que o Spark nos entregue apenas 1 arquivo salvo.
    .write
    .format("delta")    # Formato de salvamento do arquivo.
    .mode("overwrite")   # Modo de salvamento, aqui se caso ja contesse dados no Schema o "overwrite" iria subscreve-los.
    .saveAsTable(f"{catalog}.{schema}.{tablename}"))   # Aqui é onde dizemos para o Spark o caminho de salvamento, e especicamos que no Schema o arquivo será salvo como Tabela.
  
else:
  print('Tabela já existente :/')

In [0]:
%sql
SELECT * FROM bronze.olist_ecommerce.orders

In [0]:
# Nesta camada de código preciso da importação de arquivos CDC. Onde no futuro esses arquivos que precisam ser atualizados serão a principal fonte de dados para a atualização da Tabela presente no Schema Bronze. 

# Importando e fazendo a leitura de dados com o Spark.
(spark.read
    .format("csv")
    .option("header", "true")
    .load(f"/Volumes/raw/olist_ecommerce/cdc/{tablename}/")
    .createOrReplaceTempView(f"view_{tablename}"))  # Essa linha de código nos permite transformar o Dataframe que foi lido pelo Spark em uma Tabela Temporária, nos permitindo fazer consultas SQL.

query = f''' 
SELECT *
FROM view_{tablename}
QUALIFY ROW_NUMBER() OVER(PARTITION BY {id_field} ORDER BY {timestamp_field} DESC) = 1
'''     # Nesta parte do código estamos fazendo uma consulta SQL, onde tenho o objetivo de fazer um filtro com que apareça os dados mais atualizados de cada cliente presente no arquivo CDC.

df_cdc_table = spark.sql(query)    # Transformando o resultado da consulta SQL em um Dataframe.
df_cdc_table.display()

In [0]:
# Aqui é onde iremos fazer um Merge para atualizar a Tabela presente no Schema Bronze. A Tabela só contém dados Full Load desatualizados, e com os dados obtidos acima através de arquivos CDC, iremos fazer um Merge para a atualização da Tabela.

bronze = delta.DeltaTable.forName(spark, f"{catalog}.{schema}.{tablename}") # Aqui conseguimos fazer com que, conversamos com nossa tabela através de uma variável.

(bronze.alias("b")
     .merge(df_cdc_table.alias("c"), f"b.{id_field} = c.{id_field}")    # Fazendo um JOIN entre as tabelas.
     .whenMatchedDelete(condition = "c.OP = 'D'")      # Condição para deletar um dado presente na Tabela Bronze.
     .whenMatchedUpdateAll(condition = "c.OP = 'U'")   # Condição para atualizar um dado presente na Tabela Bronze
     .whenNotMatchedInsertAll(condition = "c.OP = 'I' OR c.OP = 'U'") # Condição para inserir um dado novo na Tabela Bronze.
     .execute()

)

In [0]:
%sql
SELECT * FROM bronze.olist_ecommerce.orders
where order_id = '76d3da933ea02dff5b27a64415350c37'